In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Real or Not? NLP with Disaster Tweets - How far can you get with Auto-EDA?


There are several excellent EDA notebooks against this competition where Kaggler’s have spent time exploring the data and which do a great job picking apart some of the attributes and idiosyncrasies’ that one should look out for when building models.

In this notebook I wanted to show some additional insight you can get more or less instantly by just throwing the raw data against [Kortical's new free automated EDA tool](https://kortical.com/ml-data-prep?utm_source=kaggle&utm_medium=post&utm_campaign=disaster_tweets&utm_term=free&utm_content=sign-up-beta).



## First look at the data

![First look at the data](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/first_look_at_the_data.png)

Just from the first glance at the data we can see we have 4 features and a binary target column. We can see that the location column seems to contain little in the way of validation, and has such been treated as a text field rather than as a categorical. We can also see that it has a number of missing values (which appear as 'nan' here).


## Data Overview

Lets jump straight into the data overview:

![Data Overview](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/data_overview.png)


We can see that there are 7.6k rows, and each row is roughly 130 Bytes. There are 2 text, 2 categorical and 1 numerical column. This data overview is more useful for larger datasets to get a general overview of the types of columns - a useful sense check.


## Data Completeness

The aspect we will take a look at first is data completeness - a frequent occurrence when pulling data from different sources is missing values in some columns from an incorrect join for example. This chart gives a birds-eye overview of our columns:

![Birds Eye view of data completeness](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/data_completeness_birds_eye.png)

This dataset passes with flying colours - greener bars on the right-hand chart mean lots of unique values (high cardinality), which means they contain a higher density of information, usually meaning they are more helpful in solving the problem. We are also able to see that the location column has around about 33% of its values missing though: 

![Data Completeness closeup](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/data_completeness_closeup.png)

The next step in the automated EDA process is to do a deeper dive on each of the input columns.

## Column Insights

The meat of the EDA happens when we start exploring the columns.

The first item of interest is the overall feature importances. 

![Overall Feature Importances](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/zoomed_in_feature_importance.png) 
 
This is calculated by applying an ensemble of hundreds of different models to create a model-agnostic ranking.

Unsurprisingly we see that `text` which contains the actual content of the tweet is the most important feature by a reasonable degree. As we would also expect, it shows that the `id` column is the least important. 

We saw from our data completeness chart that the `location` column has many missing values so it’s not surprising it should be low importance, we will investigate this further.

Let's start exploring the columns:


### Column insight: `text`


![Text Insight](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/text_insight.png)

We can see that this field is almost all unique (98.56%) which is what we would expect. The average number of characters per field is 101, which again seems reasonable, however, we can see that the maximum number characters is 157 which seems a bit suspect given the Twitter character limit of 140 (although this has recently been increased, it seems unlikely this dataset was pulled after that came into place).


Let's take a look at these rows in case these rows are corrupted somehow:



In [ ]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')

pd.options.display.max_colwidth = 200
over_140_chars = train[train['text'].str.len() > 140]
over_140_chars['text']

There look to be some characters which are outside the normal UTF-8 charset potentially causing this. However, this doesn't look like anything too egregious and can probably be ignored.

The wordcloud has had the most common stopwords automatically removed, although a few still remain:

![Wordcloud on tweet text](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/wordcloud.png)

On the whole, we see a high prevalence for words like *police* *fire* and *emergency* which makes sense. Beyond sense checking the field contains what we think it should, the wordcloud isn't really telling us much.

We see more interesting results if we choose the "vs Target" option. Behind the scenes this uses an ensemble of models to determine which the most important words were for predicting if a tweet referred to a real disaster:


![Tweet text vs Target](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/tweet_text_vs_target.png)


We see locations which are related to real tragedies such as *Hiroshima* or *California* (presumably due to wildfires) are almost completely predictive of class. This leads me to think that we could potentially explore creating some kind of binary indicator that is true if a real location appears as a string in the tweet text.mm




### Column insight: `keyword`

This field contains a keyword which has been extracted from the tweet text through methods unknown.

![Keyword Distribution](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/keyword_distriibution.png)

We can see that there are a total of 222 unique categories and that the majority class is actually nan (missing).

The pure distribution doesn't tell us very much until we compare it with the Target - once we look at this view, we can see that several categories are extremely predictive.:

![Keyword vs Target](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/keyword_vs_target.png)

When the keyword *outbreak* is used, it is almost always a real event, however when the term *body bags* is used, this almost always correlates to a fake event! There is a bit of a pattern here where more evocative or hyperbolic words such as *armageddon*, *fear*, *hellfire* and *deluge* seem much more likely to appear in a tweet which does not relate to a real disaster.


Maybe if we found a corpus of words from a tabloid newspaper and scored each tweet against it, we could have another interesting feature (given that tabloids are known to employ more evocative and flowery language!).



### Column insight: `location`


![Location vs Target](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/Location_vs_target.png)

This column ranks low in the overall feature importances, which often happens if there are a lot of missing values. If we look at the five most important words with respect to the target, we see that even the most important of them *york* only separates the class 70/30, which isn't very informative compared with some of the other features we have explored.


Moreover, we can see that 43% (!)  of the values are unique which suggests that there isn’t very much in the way of validation on this field. We can easily check this using `value_counts()`:


In [ ]:
train['location'].value_counts().tail(20)

It appears that “#WhereverI'mAt” and “Stay Fly?” are just a  few of the geographic locations users have helpfully entered! This will also explain why it has such low importance despite locations appearing as important words in the `text` feature. 

To make this field any use, we need to perform some serious cleaning and feature engineering. Removing all special characters and punctuation and then perhaps performing some matching against a corpus of real locations could make it more helpful! 

Once we have matched it against a geographic corpus, it might also make sense to split by different regional granularities - i.e if the location is "London", we could also have a country field which would read "UK" and maybe even a continent field reading "Europe".


### Column insight: `id`

This is the incremental row index, and shouldn't contain any information about the target, however, if we look at this distribution against the target, we can see that a model might consider certain split points such as <1.7k as containing information:

![ID vs Target](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/id_vs_target.png)

This implies that the rows were not shuffled prior to the `id` column being applied, or that the `id` column captures some implicit structure to the data, or even just that the generated `id`s correlate to the target slightly due to random chance and the small number of observations (the less observations, the more likely a column of random noise will happen to correlate with the target).

We should definitely drop this column as we can not make any guarantees that whatever real information in it (if any), it will translate to unseen data and so the model is liable to overfit quite badly on this feature.


Let move on to some automated target insights:


### Target Insights


![Target distribution](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/target_distribution.png)


We see that there are only 43% positive examples to 57% negative, making the classes slightly imbalanced (but not to any great extent). 


![Target Scores](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/target_scores.png)

The auto EDA tool actually calls out the recommended evaluation metric (AUC in this case), and also tells us that a model just predicting 0 would get an accuracy of 57%, which we can use as our absolute baseline score.


The tool actually goes further and shows what kind of machine learning f1-scores we can expect on either class:

![Target Scores Focus](https://storage.googleapis.com/kortical-marketing-content/articles/kaggle_realornot/target_scores_on_positive_class.png)


Although these haven’t had the full Kortical AutoMl treatment and so don’t suggest upper bounds by any measure, it does give us an idea what kind of F1-score we should expect on the positive class - 68%. Accept nothing less from your model!




# Summary


Although this probably isn’t likely to be the final extent of anyone’s EDA investigation, it does show that before we have even opened our notebook, we can very quickly have several important insights into the data:

* Locations as words, where they appear in the `text` column seem extremely informative at spotting real disasters
* However, the `location` field itself is a mess, requires serious data cleaning and feature engineering to increase its value to the model 
* Keywords which have flowery language are good at spotting non-real disaster tweets, which gave us some ideas for feature engineering.
* The `id` field should almost certainly be removed
* We shouldn’t accept anything less than .68 F1-score on the positive class!



This EDA was done using Korticals free EDA tool, available [here](https://kortical.com/ml-data-prep?utm_source=kaggle&utm_medium=post&utm_campaign=disaster_tweets&utm_term=free&utm_content=sign-up-beta)